#  Unit 2.4a & 2.4b Hacks
- toc: true 
- badges: true
- comments: true
- categories: [markdown]

In [42]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///files/sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


In [52]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash

''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the Prop class to manage actions in the 'Props' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) Prop represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class PropPicker(db.Model): #Class Prop is defining the template for Props. It is a class definition template. 
    # We are going to use this templete to create objetcs of type Props.
    # This code is not going to do anything right away, but we are going to use this code later to create objects
    # (db.Model) is used as a template for the data base.
    __tablename__ = 'Props'  # table name is plural, class name is singular

    # Define the Prop schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _cthit = db.Column(db.String(255), unique=True, nullable=False)

    # constructor of a Prop object, initializes the instance variables within object (self)
    # This init method enables us to create an object from our Prop class
    def __init__(self, name, cthit):
        self._name = name    # variables with self prefix become part of the object, 
        self._cthit = cthit

    # a name getter method, extracts name from object
    # Setters enable us to change the atribute (if you put the wrong dob you can reset it with the setter)
    # Getters (Property) enable us to get information out of the object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts cthit from object
    @property
    def cthit(self):
        return self._cthit
    
    # a setter function, allows cthit to be updated after initial object creation
    @cthit.setter
    def cthit(self, cthit):
        self._cthit = cthit
        
    # check if cthit parameter matches Prop id in object, return boolean
    def is_cthit(self, cthit):
        return self._cthit == cthit

    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from Prop(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Props table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "cthit": self.cthit,
        }

    # CRUD update: updates Prop name, password, phone
    # returns self
    def update(self, name="", cthit=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(cthit) > 0:
            self.cthit = cthit
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
"""Database Creation and Testing """


# Builds working data for testing
def initprops():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = PropPick(name='Nikola Jokic', cthit='97%')
        u2 = PropPick(name='Kelly Oubre Jr.', cthit='100%')
        u3 = PropPick(name='Draymond Green', cthit='34%')
        u4 = PropPick(name='Clint Capela', cthit='92%')
        u5 = PropPick(name='Xavier Tillman', cthit='90%')
        u6 = PropPick(name='Luka Doncic', cthit='87%')


        PropPick = [u1, u2, u3, u4, u5, u6]

        """Builds sample prop/note(s) data"""
        for prop in PropPick:
            try:
                '''add prop to table'''
                object = prop.create()
                print(f"Created new cthit {object.cthit}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist cthit {prop.cthit}, or error.")

InvalidRequestError: Table 'Props' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [ ]:
# SQLAlchemy extracts single prop from database matching prop ID
def find_by_cthit(cthit):
    with app.app_context():
        prop = prop.query.filter_by(cthit=cthit).first()
    return prop # returns prop object

# Check credentials by finding prop and verify password
def check_credentials(cthit, name):
    # query email and return prop record
    prop = find_by_cthit(cthit)
    if prop == None:
        return False
    if name == None:
        return True
    return False
        
check_credentials("87%", "Luka Doncic")

In [ ]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize prop time to see if cthit exists
    cthit = input("Enter your prop id:")
    prop = find_by_cthit(cthit)
    try:
        print("Found\n", prop.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    cthit = input("Enter your cthit")
    
    # Initialize prop object before date
    prop = prop(name=name, 
                cthit=cthit, 
                )
           
    # write object to database
    with app.app_context():
        try:
            object = prop.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error cthit {cthit}")
        
create()

In [ ]:
# SQLAlchemy extracts all props from database, turns each prop into JSON
def read():
    with app.app_context():
        table = Prop.query.all()
    json_ready = [prop.read() for prop in table] # "List Comprehensions", for each prop add prop.read() to list
    return json_ready

read()